# Práce s metadaty v Národním katalogu otevřených dat

[Národní katalog otevřených dat (NKOD)](https://data.gov.cz/datové-sady) je centrálním místem, kde jsou evidována metadata o veškerých otevřených datech poskytovaných veřejnou správou ČR. Katalogizační záznamy jsou strukturovány dle slovníku [DCAT-AP](https://joinup.ec.europa.eu/solution/dcat-application-profile-data-portals-europe?f%5B0%5D=solution_content_bundle%3Aasset_release).

V tomto Jupyter Notebooku ukazujeme možnosti práce s katalogizačními záznamy v NKOD prostřednictvím strojově zpracovatelného API.

Zdrojový kód tohoto Jupyter Notebooku lze [stáhnout](https://github.com/opendata-mvcr/opendata-mvcr.github.io/blob/master/datové-články/národní-katalog-otevřených-dat/nkod.ipynb) z GitHubu.

## Konceptuální schéma DCAT-AP

Následující obrázek ukazuje konceptuální schéma DCAT-AP, ptodle kterého jsou katalogizační záznamy v NKOD strukturovány. Jeho detailní vysvětlení naleznete ve [specifikaci DCAT-AP](https://joinup.ec.europa.eu/solution/dcat-application-profile-data-portals-europe?f%5B0%5D=solution_content_bundle%3Aasset_release).

![Konceptuální schéma DCAT-AP](https://joinup.ec.europa.eu/sites/default/files/distribution/access_url/2019-05/ef3c9392-e38a-40d0-8bf0-b5a847ffae49/DCAT-AP_1.2.1.PNG)

## Zpřístupnění dat z NKOD

Metadata jsou dostupná ve strojově čitelné podobě jako otevřená data ve dvou podobách - v podobě CSV souborů a v podobě datového modelu [RDF](https://www.w3.org/RDF/).

Pro získání CSV podoby slouží 2 datové sady: [Datové sady z Národního katalogu otevřených dat](https://data.gov.cz/datov%C3%A1-sada?iri=https%3A%2F%2Fdata.gov.cz%2Fzdroj%2Fdatov%C3%A9-sady%2FMV%2F670509643) a [Distribuce z Národního katalogu otevřených dat](https://data.gov.cz/datov%C3%A1-sada?iri=https%3A%2F%2Fdata.gov.cz%2Fzdroj%2Fdatov%C3%A9-sady%2FMV%2F670509648).

Pro získání RDF podoby slouží datová sada [Národní katalog otevřených dat](https://data.gov.cz/datov%C3%A1-sada?iri=https%3A%2F%2Fdata.gov.cz%2Fzdroj%2Fdatov%C3%A9-sady%2FMV%2F670509543). RDF podoba je navržena dle principů [otevřených propojených dat](https://www.lod-cloud.net/) (angl. Linked Open Data, zkr. LOD). NKOD také nabízí veřejně dostupný [SPARQL endpoint](t). SPARQL endpoint je webová služba, která přijímá dotazy v dotazovacím jazyku [SPARQL](https://www.w3.org/TR/sparql11-overview/) a výsledky vrací buď v podobě tabulek serializovaných jako CSV, JSON nebo XML soubory nebo ve zvolené serializaci datového modelu [RDF](https://www.w3.org/RDF/).

V tomto článku ukazujeme několik příkladů v dotazovacím jazyku SPARQL, které můžete využít pro strojové vyhledávání datových sad v NKOD a jejich distribucí. Prostřednictvím HTTPS GET/POST požadavků můžete na [SPARQL endpoint](https://data.gov.cz/sparql) zasílat SPARQL dotazy dle příkladů SPARQL dotazů uvedených níže. V případě SELECT dotazů jsou výsledkem tabulková data, která si můžete nechat vracet v různých formátech (CSV, XML, JSON, ...). Seznam možných formátů a způsob jak se na daný formát zeptat zjistíte pohledem na kód formulářového pole "Results Format" v [uživatelském rozhraní SPARQL endpointu](https://data.gov.cz/sparql).

Alternativou je zprovoznit si vlastní lokální RDF úložiště a využít jeho vlastní SPARQL endpoint pro dotazování. Lze využít např. [Apache Jena Fuseki](https://jena.apache.org/documentation/fuseki2/) nebo [GraphDB](http://graphdb.ontotext.com/). V Národním katalogu otevřených dat je potřeba získat odkazy ke stažení JSON-LD distribucí.

Pro spouštění SPARQL dotazů uvedených v tomto článku prostřednictvím prostřeí [Jupyter Notebooks](https://jupyter.org/) je nejprve nutno nastavit prostředí pro prácí s RDF a SPARQLem.

In [1]:
import  sys
from  gastrodon  import RemoteEndpoint,QName,ttl,URIRef,inline
import pandas as pd
pd.options.display.width=120
pd.options.display.max_colwidth=100

In [2]:
prefixes=inline("""
    @prefix dcat: <http://www.w3.org/ns/dcat#> .
    @prefix dct: <http://purl.org/dc/terms/> .
""").graph

In [3]:
endpoint=RemoteEndpoint(
    "https://data.gov.cz/sparql"
    ,default_graph=""
    ,prefixes=prefixes
    ,base_uri=""
)

Následující dotaz vrátí seznam datových sad poskytovaných daným poskytovatelem (zde Ministerstvem vnitra ČR)

In [4]:
datasetyPoskytovatel=endpoint.select("""PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?dataset ?datasetNazev
WHERE {
  ?dataset a dcat:Dataset ;
    dct:title ?datasetNazev ;
    dct:publisher ?poskytovatel .

  VALUES ?poskytovatel {<https://data.gov.cz/zdroj/ovm/00007064>}
}""")
datasetyPoskytovatel

,dataset,datasetNazev
0,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-ks-...,Komanditní společnost - Platný (Ostrava - 2010)
1,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-ust...,Ústav - Platný (Praha - 2010)
2,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-osz...,Organizační složka zahraniční nadace - Úplný (Praha - 2017)
3,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-ks-...,Komanditní společnost - Úplný (Hradec Králové - 2014)
4,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-svj...,Společenství vlastníků jednotek - Platný (Ústí nad Labem - 2016)
5,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-zah...,Zahraniční fyzická osoba - Platný (Brno - 2014)
6,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-vos...,Veřejná obchodní společnost - Platný (Hradec Králové - 2018)
7,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-pob...,Pobočný spolek - Úplný (Ostrava - 2015)
8,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-odb...,Odborová organizace - Platný (Brno - 2011)
9,https://data.gov.cz/zdroj/datové-sady/https---dataor.justice.cz-api-3-action-package_show-id-nad...,Nadace - Platný (Ústí nad Labem - 2014)


Ve výsledku vidíme pro každou datovou sadu její IRI a název. Pozor ale, IRI není odkazem do NKODu. Jedná se o identifikační IRI přiřazené datové sadě dle principů LOD. IRI jednak datovou sadu identifikuje a zároveň je dereferencovatelné, tj. prostřednictvím prototkolu HTTP/HTTPS lze přistoupit k datům o entitě. Data jsou poskytnuta v modelu RDF. URL odkaz na zobrazení katalogizačního záznamu v NKOD má tvar ```https://data.gov.cz/datová-sada?iri=[IRI]```, kde ```[IRI]``` je výše popsané IRI datové sady.

Další dotaz vrátí seznam datových sad opatřených daným klíčovým slovem ("registr práv a povinností")

In [5]:
datasetyKlíčovéSlovo=endpoint.select("""PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?dataset ?datasetNazev
WHERE {
  ?dataset a dcat:Dataset ;
    dct:title ?datasetNazev ;
    dcat:keyword ?slovo .
  
  FILTER (STR(?slovo)="registr práv a povinností")
}""")
datasetyKlíčovéSlovo

,dataset,datasetNazev
0,https://data.gov.cz/zdroj/datové-sady/MV/706529437/cf79f7477c758122a8d99891e66a9800,Číselník typů místních příslušností k subjektům
1,https://data.gov.cz/zdroj/datové-sady/MV/706529437/02925784ee989ac89fbb61d2e3b94c8e,Úkony
2,https://data.gov.cz/zdroj/datové-sady/MV/706529437/9c73b802263c5e0ccf5542f10fbc35bb,Agendy
3,https://data.gov.cz/zdroj/datové-sady/MV/706529437/527b4095146db54b65c9eca6b969b6ab,Role
4,https://data.gov.cz/zdroj/datové-sady/MV/706529437/735d2f4f35c2493468ef682afd441418,Kategorie soukromoprávních uživatelů údajů
5,https://data.gov.cz/zdroj/datové-sady/MV/706529437/663bce4f3fef2f95fed0328c0e335b0e,Číselník typů rozhodnutí o schválení převodu působnosti
6,https://data.gov.cz/zdroj/datové-sady/MV/706529437/281575084358cb8be628b7cc66020c2c,Číselník typů subjektů
7,https://data.gov.cz/zdroj/datové-sady/MV/706529437/77acdd8bb9c7bad994c58a9e4764c5a2,Číselník typů rozhodnutí o založení veřejnoprávní smlouvy
8,https://data.gov.cz/zdroj/datové-sady/MV/706529437/466dbfa43437d4847ea45b0b082aa80b,Číselník typů činností
9,https://data.gov.cz/zdroj/datové-sady/MV/706529437/420dc4fe7034cdb75bb0d1baf2c357d3,Číselník elektronických forem úkonů


Další dotaz vrátí seznam datových sad s tématem "Vláda a veřejný sektor"

In [6]:
datasetyTéma=endpoint.select("""PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?dataset ?datasetNazev
WHERE {
  ?dataset a dcat:Dataset ;
    dct:title ?datasetNazev ;
    dcat:theme ?tema .
  
  VALUES ?tema {<http://publications.europa.eu/resource/authority/data-theme/GOVE>}
}""")
datasetyTéma

,dataset,datasetNazev
0,https://data.gov.cz/zdroj/datové-sady/mpo/694709350,Seznam OSS a PO Ministerstva prumyslu a obchodu
1,https://data.gov.cz/zdroj/datové-sady/MV/706529437/cf79f7477c758122a8d99891e66a9800,Číselník typů místních příslušností k subjektům
2,https://data.gov.cz/zdroj/datové-sady/MV/706529437/02925784ee989ac89fbb61d2e3b94c8e,Úkony
3,https://data.gov.cz/zdroj/datové-sady/MV/706529437/9c73b802263c5e0ccf5542f10fbc35bb,Agendy
4,https://data.gov.cz/zdroj/datové-sady/MV/706529437/70602dcd9f72c78f53ab8b6da04eafc1,Přístupy k agendám CzechPOINT - 2014
5,https://data.gov.cz/zdroj/datové-sady/MV/706529437/527b4095146db54b65c9eca6b969b6ab,Role
6,https://data.gov.cz/zdroj/datové-sady/MV/706529437/735d2f4f35c2493468ef682afd441418,Kategorie soukromoprávních uživatelů údajů
7,https://data.gov.cz/zdroj/datové-sady/MV/706529437/663bce4f3fef2f95fed0328c0e335b0e,Číselník typů rozhodnutí o schválení převodu působnosti
8,https://data.gov.cz/zdroj/datové-sady/MV/706529437/4f734ac2f7d6465da8f5dfb6130ef98b,Přístupy k agendám CzechPOINT - 2015
9,https://data.gov.cz/zdroj/datové-sady/MV/706529437/281575084358cb8be628b7cc66020c2c,Číselník typů subjektů


Můžete si všimnout, že zatímco klíčová slova jsou v podobě řetězců, tak témata jsou entity s identifikačním IRI.

V NKOD jsou dle pravidel DCAT-AP uvažovány 2 sady témat. První sada témat je obecnou sadou témat, která používá [Evropský datový portál](https://www.europeandataportal.eu/) pro hrubou klasifikaci datových sad na své hlavní stránce. Seznam těchto témat vrací následující dotaz.

In [7]:
témataEDP=endpoint.select("""PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?tema ?temaNazevCS ?temaNazevEN
WHERE {
  ?tema skos:inScheme <http://publications.europa.eu/resource/authority/data-theme> ;
    skos:prefLabel ?temaNazevCS, ?temaNazevEN .
  FILTER(LANG(?temaNazevCS)="cs")
  FILTER(LANG(?temaNazevEN)="en")
}""")
témataEDP

,tema,temaNazevCS,temaNazevEN
0,http://publications.europa.eu/resource/authority/data-theme/AGRI,"Zemědělství, rybolov, lesnictví a výživa","Agriculture, fisheries, forestry and food"
1,http://publications.europa.eu/resource/authority/data-theme/ECON,Hospodářství a finance,Economy and finance
2,http://publications.europa.eu/resource/authority/data-theme/EDUC,"Vzdělávání, kultura a sport","Education, culture and sport"
3,http://publications.europa.eu/resource/authority/data-theme/ENER,Energie,Energy
4,http://publications.europa.eu/resource/authority/data-theme/ENVI,Životní prostředí,Environment
5,http://publications.europa.eu/resource/authority/data-theme/GOVE,Vláda a veřejný sektor,Government and public sector
6,http://publications.europa.eu/resource/authority/data-theme/HEAL,Zdraví,Health
7,http://publications.europa.eu/resource/authority/data-theme/INTR,Mezinárodní otázky,International issues
8,http://publications.europa.eu/resource/authority/data-theme/JUST,"Spravedlnost, právní systém a veřejná bezpečnost","Justice, legal system and public safety"
9,http://publications.europa.eu/resource/authority/data-theme/OP_DATPRO,Předběžné údaje,Provisional data


Druhou sadou témat je slovník [Eurovoc](https://publications.europa.eu/en/web/eu-vocabularies/th-dataset/-/resource/dataset/eurovoc). Seznam témat ze slovníku Eurovoc, použitých pro datové sady v NKOD vrací tento dotaz:

In [8]:
témataEurovoc=endpoint.select("""PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?kodTematu ?tema ?temaNazevCS ?temaNazevEN
WHERE {
  ?tema skos:inScheme <http://eurovoc.europa.eu/100141> ;
    skos:prefLabel ?temaNazevCS, ?temaNazevEN ;
    skos:notation ?kodTematu .
  FILTER(LANG(?temaNazevCS)="cs")
  FILTER(LANG(?temaNazevEN)="en")
  
  ?dataset dcat:theme ?tema .
}
ORDER BY ?kodTematu""")
témataEurovoc

,kodTematu,tema,temaNazevCS,temaNazevEN
0,1005,http://eurovoc.europa.eu/1005,financování EU,EU financing
1,1017,http://eurovoc.europa.eu/1017,finance místních úřadů,local authority finances
2,1018,http://eurovoc.europa.eu/1018,veřejné finance,public finance
3,1063,http://eurovoc.europa.eu/1063,les,forest
4,1231,http://eurovoc.europa.eu/1231,zdravotně postižený člověk,disabled person
5,1317,http://eurovoc.europa.eu/1317,místní daň,local tax
6,1334,http://eurovoc.europa.eu/1334,pracovní nezpůsobilost,incapacity for work
7,1392,http://eurovoc.europa.eu/1392,knižní průmysl,book trade
8,1422,http://eurovoc.europa.eu/1422,informace,information
9,1448,http://eurovoc.europa.eu/1448,kontrolní úřad,supervisory body


Předchozí dotazy vracely seznam datových sad. Každá datová sada má 1 nebo více distribucí, kdy každá distribuce je materializací datové sady ve strojově čitelné podobě, ke které je možno přístoupit, např. ji stáhnout v podobě datového souboru. 

Následující dotaz je rozšířením výše uvedeného dotazu na datové sady daného poskytovatele (Ministerstvo vnitra) o dotaz na distribuce těchto datových sad.

In [9]:
distribucePoskytovatel=endpoint.select("""PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?dataset ?datasetNazev ?distribuce ?urlKeStazeni ?urlProPristup ?mediaType ?format
WHERE {
  ?dataset a dcat:Dataset ;
    dct:title ?datasetNazev ;
    dct:publisher ?poskytovatel ;
    dcat:distribution ?distribuce .
   
  OPTIONAL {
    ?distribuce dcat:downloadURL ?urlKeStazeni .
  }
  
  OPTIONAL {
    ?distribuce dcat:accessURL ?urlProPristup .
  }
  
  OPTIONAL {
    ?distribuce dcat:mediaType ?mediaType .
  }
  
  OPTIONAL {
    ?distribuce dct:format ?format .
  }

  VALUES ?poskytovatel {<https://data.gov.cz/zdroj/ovm/00007064>}
}
ORDER BY ?dataset""")
distribucePoskytovatel

,dataset,datasetNazev,distribuce,urlKeStazeni,urlProPristup,mediaType,format
0,https://data.gov.cz/zdroj/datové-sady/MV/143188351,Nepotřebné nemovité věci,https://data.gov.cz/zdroj/datové-sady/MV/143188351/distribuce/1878df68ace3c30dcc67cd95e6318f81,None,https://portal.gov.cz/portal/rejstriky/data/115217898/index.xml,http://www.iana.org/assignments/media-types/text/xml,http://publications.europa.eu/resource/authority/file-type/XML
1,https://data.gov.cz/zdroj/datové-sady/MV/143188372,Smlouvy v Registru smluv,https://data.gov.cz/zdroj/datové-sady/MV/143188372/distribuce/3f69e540073da6a899e924f7c9bf9483,None,https://data.smlouvy.gov.cz/,http://www.iana.org/assignments/media-types/text/xml,http://publications.europa.eu/resource/authority/file-type/XML
2,https://data.gov.cz/zdroj/datové-sady/MV/170966154,Seznam datových schránek,https://data.gov.cz/zdroj/datové-sady/MV/170966154/distribuce/e83fd2bc728a95af96c3bb121289ed78,None,https://seznam.gov.cz/ovm/datafile.do?format=xml&service=seznam_ds_ovm,http://www.iana.org/assignments/media-types/text/xml,http://publications.europa.eu/resource/authority/file-type/XML
3,https://data.gov.cz/zdroj/datové-sady/MV/170966154,Seznam datových schránek,https://data.gov.cz/zdroj/datové-sady/MV/170966154/distribuce/82cf17be58526bd8d27dca87f833981d,None,https://seznam.gov.cz/ovm/datafile.do?format=xml&service=seznam_ds_fo,http://www.iana.org/assignments/media-types/text/xml,http://publications.europa.eu/resource/authority/file-type/XML
4,https://data.gov.cz/zdroj/datové-sady/MV/170966154,Seznam datových schránek,https://data.gov.cz/zdroj/datové-sady/MV/170966154/distribuce/57c914e16ef79f0bcbab13e9a85c8ac3,None,https://seznam.gov.cz/ovm/datafile.do?format=xml&service=seznam_ds_po,http://www.iana.org/assignments/media-types/text/xml,http://publications.europa.eu/resource/authority/file-type/XML
5,https://data.gov.cz/zdroj/datové-sady/MV/170966154,Seznam datových schránek,https://data.gov.cz/zdroj/datové-sady/MV/170966154/distribuce/e1d6cf0246ce97081903cfc65c826b0f,None,https://seznam.gov.cz/ovm/datafile.do?format=xml&service=seznam_ds_pfo,http://www.iana.org/assignments/media-types/text/xml,http://publications.europa.eu/resource/authority/file-type/XML
6,https://data.gov.cz/zdroj/datové-sady/MV/240415378,Nestátní neziskové organizace ČR - archiv ze dne 10.04.2015,https://data.gov.cz/zdroj/datové-sady/MV/240415378/distribuce/cb6d55e486b76ba32ce821e1015def68,http://www.isnno.cz/opendata/enno_201504.zip,http://www.isnno.cz/opendata/enno_201504.zip,http://www.iana.org/assignments/media-types/application/xml,http://publications.europa.eu/resource/authority/file-type/XML
7,https://data.gov.cz/zdroj/datové-sady/MV/246534435,Nestátní neziskové organizace ČR - archiv ze dne 01.06.2017,https://data.gov.cz/zdroj/datové-sady/MV/246534435/distribuce/d91c4aac4ab210f1df7021ecc0fec58b,http://www.isnno.cz/opendata/enno_201705.zip,http://www.isnno.cz/opendata/enno_201705.zip,http://www.iana.org/assignments/media-types/application/xml,http://publications.europa.eu/resource/authority/file-type/XML
8,https://data.gov.cz/zdroj/datové-sady/MV/254676116,Nestátní neziskové organizace ČR - archiv ze dne 01.09.2017,https://data.gov.cz/zdroj/datové-sady/MV/254676116/distribuce/1b39f1187879178e04c1b4375c7ef6c8,http://www.isnno.cz/opendata/enno_201708.zip,http://www.isnno.cz/opendata/enno_201708.zip,http://www.iana.org/assignments/media-types/application/xml,http://publications.europa.eu/resource/authority/file-type/XML
9,https://data.gov.cz/zdroj/datové-sady/MV/255131823,Obsazovaná služební místa,https://data.gov.cz/zdroj/datové-sady/MV/255131823/distribuce/c0bc409c6a9e3818feceff3890eb7d61,https://portal.isoss.cz/opendata/ISoSS_Opendata_EOSM.xml,https://portal.isoss.cz/opendata/ISoSS_Opendata_EOSM.xml,http://www.iana.org/assignments/media-types/text/xml,http://publications.europa.eu/resource/authority/file-type/XML


Filtry lze různě kombinovat. Následující dotaz vrací JSON distribuce datových sad z Registru práv a povinností.

In [10]:
distribuceRPPJSON=endpoint.select("""PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dct: <http://purl.org/dc/terms/>

SELECT ?dataset ?datasetNazev ?urlKeStazeni ?urlProPristup ?format 
WHERE {
  ?dataset a dcat:Dataset ;
    dct:title ?datasetNazev ;
    dcat:keyword ?slovo ;
    dcat:distribution ?distribuce .
   
  OPTIONAL {
    ?distribuce dcat:downloadURL ?urlKeStazeni .
  }
  
  OPTIONAL {
    ?distribuce dcat:accessURL ?urlProPristup .
  }
  
  OPTIONAL {
    ?distribuce dct:format ?format .
  }
  
  ?distribuce dcat:mediaType <http://www.iana.org/assignments/media-types/application/json> .
  
  FILTER (STR(?slovo)="registr práv a povinností")
}""")
distribuceRPPJSON

,dataset,datasetNazev,urlKeStazeni,urlProPristup,format
0,https://data.gov.cz/zdroj/datové-sady/MV/706529437/cf79f7477c758122a8d99891e66a9800,Číselník typů místních příslušností k subjektům,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_prislusnost_subjektu.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_prislusnost_subjektu.json,http://publications.europa.eu/resource/authority/file-type/JSON
1,https://data.gov.cz/zdroj/datové-sady/MV/706529437/02925784ee989ac89fbb61d2e3b94c8e,Úkony,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/ukony.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/ukony.json,http://publications.europa.eu/resource/authority/file-type/JSON
2,https://data.gov.cz/zdroj/datové-sady/MV/706529437/9c73b802263c5e0ccf5542f10fbc35bb,Agendy,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/agendy.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/agendy.json,http://publications.europa.eu/resource/authority/file-type/JSON
3,https://data.gov.cz/zdroj/datové-sady/MV/706529437/527b4095146db54b65c9eca6b969b6ab,Role,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/role.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/role.json,http://publications.europa.eu/resource/authority/file-type/JSON
4,https://data.gov.cz/zdroj/datové-sady/MV/706529437/735d2f4f35c2493468ef682afd441418,Kategorie soukromoprávních uživatelů údajů,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/kategorie_spuu.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/kategorie_spuu.json,http://publications.europa.eu/resource/authority/file-type/JSON
5,https://data.gov.cz/zdroj/datové-sady/MV/706529437/663bce4f3fef2f95fed0328c0e335b0e,Číselník typů rozhodnutí o schválení převodu působnosti,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_rozhodnuti_schvaleni.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_rozhodnuti_schvaleni.json,http://publications.europa.eu/resource/authority/file-type/JSON
6,https://data.gov.cz/zdroj/datové-sady/MV/706529437/281575084358cb8be628b7cc66020c2c,Číselník typů subjektů,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_typ_subjektu.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_typ_subjektu.json,http://publications.europa.eu/resource/authority/file-type/JSON
7,https://data.gov.cz/zdroj/datové-sady/MV/706529437/77acdd8bb9c7bad994c58a9e4764c5a2,Číselník typů rozhodnutí o založení veřejnoprávní smlouvy,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_rozhodnuti_zaklad.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_rozhodnuti_zaklad.json,http://publications.europa.eu/resource/authority/file-type/JSON
8,https://data.gov.cz/zdroj/datové-sady/MV/706529437/466dbfa43437d4847ea45b0b082aa80b,Číselník typů činností,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_typ_cinnosti.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_typ_cinnosti.json,http://publications.europa.eu/resource/authority/file-type/JSON
9,https://data.gov.cz/zdroj/datové-sady/MV/706529437/420dc4fe7034cdb75bb0d1baf2c357d3,Číselník elektronických forem úkonů,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_el_forem_ukonu.json,https://rpp-opendata.egon.gov.cz/odrpp/datovasada/cis_el_forem_ukonu.json,http://publications.europa.eu/resource/authority/file-type/JSON
